In [0]:
pip install bokeh

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
ct = sc.textFile("FileStore/tables/clinicaltrial_2021.csv")
header = ct.first()
clinicaltrial_2021 = ct.filter(lambda x: x!=header)

In [0]:
print(header)

Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions


In [0]:
mh = sc.textFile("FileStore/tables/mesh.csv")
header = mh.first()
mesh = mh.filter(lambda x: x != header)
mesh.take(3)

Out[14]: ['Calcimycin,D03.633.100.221.173',
 'A-23187,D03.633.100.221.173',
 'Temefos,D02.705.400.625.800']

In [0]:
pa = sc.textFile("FileStore/tables/pharma.csv")
header = pa.first()
pharma = pa.filter(lambda x: x != header)

In [0]:
clinicaltrial_2021.take(5)

Out[16]: ['Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions',
 'NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||',
 'NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|',
 'NCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|',
 'NCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|']

In [0]:
## Question 1
clinicaltrial_2021.distinct().count()

Out[17]: 387262

In [0]:
##Question2
clinicaltrial_2021.map(lambda x: (x.split('|')[5],1)).reduceByKey(lambda x,y: x+y).takeOrdered(5, key = lambda x: -x[1])

Out[18]: [('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69),
 ('Type', 1)]

In [0]:
##Question 3
exploded_condition = clinicaltrial_2021.map(lambda x: (x.split('|')[7])).filter(lambda x: x!='').flatMap(lambda x:x.split(','))

exploded_condition.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).takeOrdered(5, key = lambda x: -x[1])

Out[19]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

In [0]:
##Question 4
ec = exploded_condition.map(lambda x: (x,1))
mesh_root = mesh.map(lambda x: x.split(',')).map(lambda x: (x[0],x[1].split('.'))).map(lambda x: (x[0],x[1][0]))
mesh_join = ec.join(mesh_root)
mesh_join.take(5)

mesh_join.map(lambda x:(x[1][1],1)).reduceByKey(lambda x,y: x+y).takeOrdered(5, key = lambda x: -x[1])

Out[20]: [('C04', 143994),
 ('C23', 136079),
 ('C01', 106674),
 ('C14', 94523),
 ('C10', 92310)]

In [0]:
parentCompany = pharma.map(lambda x: x.split(',')[1]).map(lambda x: x.replace('"',''))
#parentCompany.take(5)
sponsor = clinicaltrial_2021.map(lambda x: x.split('|')[1])
#sponsor.take(5)

sponsor.subtract(parentCompany).map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y).takeOrdered(10, key = lambda x: -x[1])


Out[21]: [('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

In [0]:
trial = clinicaltrial_2021.map(lambda x: x.split('|')).map(lambda x: (x[2],x[4]))
trial_1 = trial.filter(lambda x: 'Completed' in x).map(lambda x: x[1]).filter(lambda x: '2021' in x)
trial_2021 = trial_1.map(lambda x: (x,1)).reduceByKey(lambda x,y: x+y)
trial_2021.collect()

Out[22]: [('Jan 2021', 1131),
 ('Jun 2021', 1094),
 ('Aug 2021', 700),
 ('Apr 2021', 967),
 ('Mar 2021', 1227),
 ('May 2021', 984),
 ('Feb 2021', 934),
 ('Jul 2021', 819),
 ('Oct 2021', 187),
 ('Sep 2021', 528)]

In [0]:
trial_2021.map(lambda x: x[0]).take(3)

Out[23]: ['Jan 2021', 'Jun 2021', 'Aug 2021']

In [0]:
from datetime import datetime
def getMonthNum(monthName):
    name = monthName[0:3]
    try:
        date = datetime.strptime(name,"%b")
        month_number = date.month
        return month_number
    except:
        return 
    

In [0]:
monthly_report = trial_2021.map(lambda x:(x[0], getMonthNum(x[0]),x[1])).sortBy(lambda x: x[1]).map(lambda x: (x[0].split(' ')[0],x[2]))
monthly_report.collect()

Out[25]: [('Jan', 1131),
 ('Feb', 934),
 ('Mar', 1227),
 ('Apr', 967),
 ('May', 984),
 ('Jun', 1094),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]

In [0]:
month = monthly_report.map(lambda x: x[0]).collect()
frequency = monthly_report.map(lambda x: x[1]).collect()

In [0]:
from bokeh.embed import file_html
from bokeh.plotting import figure
from bokeh.resources import CDN

plot = figure(x_range= month, height=350, title="Monthly Studies Completion Report")
plot.vbar(x=month, top=frequency, width=0.5)
plot.xaxis.axis_label = 'month'
plot.yaxis.axis_label = 'total_Completed'
html = file_html(plot,CDN,"plot")
displayHTML(html)

<!DOCTYPE html>
 
 
 
 
 
 plot